In [1]:
import requests as re
import uri
import telebot
import urllib.request
from pathlib import Path
import urllib.request as urllib2
import cv2
import pytesseract
import logging
from PIL import Image, ImageEnhance, ImageFilter
import matplotlib.pyplot as plt
import numpy as np
import os
import time

In [2]:
start_message_text = "Hi! This is a bot made for the final ML project.\nTeam members: Suleimen Daukishov, Nurmukhammed Abeuov, Asylkhan Otegaliev. \nJust send the bot a photo and it will send you the text on it!"
BOT_TOKEN = 'BOT_TOKEN'

In [3]:
def show_image(img, title=""):
    plt.title(title)
    plt.imshow(img)
    plt.show()
    
def plot_images(imgs):
    imgs.reverse()
    plt.figure(figsize=(7, 15)) # specifying the overall grid size
    for i, img in enumerate(imgs):
        plt.subplot(3, len(imgs), i + 1)    # the number of images in the grid is 5*5 (25)
        plt.title(img[1])
        plt.imshow(img[0])
    plt.show()
    
def resize_img(img, basewidth):
    print(img)
    wpercent = (basewidth / float(img.size[0]))
    hsize = int((float(img.size[1]) * float(wpercent)))
    img = img.resize((basewidth, hsize), Image.ANTIALIAS)
    return img

def not_correct(pixel_array, percent):
    pixel_array = pixel_array.flatten()
    white_pixs = [x for x in pixel_array if x == 255]
    
    true_percent = len(white_pixs) / len(pixel_array) * 100
    
    if true_percent > percent:
        return False
    
    return True

def delete_punct(img):
    h, w = img.size
    if h < 14 and w < 14:
        return True
    return False

def get_concat_h(im1, im2):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

def get_concat_v(im1, im2):
    basewidth = im2.size[0]
    wpercent = (basewidth/float(im1.size[0]))
    im1 = im1.resize((basewidth, im1.size[1]), Image.ANTIALIAS)
    dst = Image.new('RGB', (im2.width, im1.height + im2.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst

def split_word_to_letter(path, letter_padding=1):
    img = path

    im = cv2.imread(img,0)
    pil_img = Image.open(img)
    hlimg, wlimg = im.shape[0], im.shape[1]

    ret,thresh1 = cv2.threshold(im, 127, 100, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(thresh1,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
#     show_image(im)

    image_data = pytesseract.image_to_boxes(im, output_type=pytesseract.Output.DICT, lang="kaz")
    image_data['char'] = [x.lower() for x in image_data['char']]

    # COUNT I LETTER TO EARLY STOP
    count_i = image_data['char'].count('й')

    # let_indeсes = [n for n,x in enumerate(image_data['char']) if x.lower()=='й']
    # for let_index in let_indeсes:
    #     image_data['char'].insert(let_index+1, 'й')


    # DOUBLE ы
    let_indeсes = [n for n,x in enumerate(image_data['char']) if x.lower()=='ы']
    for let_index in let_indeсes:
        image_data['char'].insert(let_index+1, 'ы')

    letters_count = len(image_data['char'])

    i = 0
    j = 0
    letter_idx = 0

    merge_list = []
    result = []
    x, y, w, h = cv2.boundingRect(contours[-1])
    i_hat = pil_img.crop((x - letter_padding, y - letter_padding, x+w + letter_padding, y+h + letter_padding))


    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
    #     print(x, y, w, h)
        # BOUND IMAGES
        img_croped_letters = pil_img.crop((x - 1, y - 1, x+w + 1, y+h + 1))

        # CHECK CORRECT IMAGES
        if not_correct(np.asarray(img_croped_letters)[-1], 10):
            continue

        # DELETE PUNCTUATIONS
        if delete_punct(img_croped_letters):
            continue

        try:
        # MERGE Ы
            if image_data['char'][-i-1] == 'ы':
                merge_list.append(img_croped_letters)
                if len(merge_list) == 2:
                    img_croped_letters = get_concat_h(merge_list[1], merge_list[0])
                    merge_list = []

        # MERGE Й
            if image_data['char'][-i-1] == 'й':
                img_croped_letters = get_concat_v(i_hat, img_croped_letters)

        except:
            break

        i = i + 1
        if len(merge_list) == 1:
            continue

        letter_idx = letter_idx + 1
        if letter_idx - 1 == letters_count:
            break
            
        result.append([img_croped_letters, image_data['char'][-i]])
    return result

In [4]:
def preprocess_photo(file_path):
    # SCRIPT
    path_image = file_path # IMG PATH

    word_padding = 5 # WORD PADDING IN PIXELS
    letter_padding = 1 # LETTER PADDING IN PIXELS

    # OPEN IMAGE 
    img = Image.open(path_image)
    show_image(img, "Original image")

    # GRAYSCALE
    img = img.convert('L')
    show_image(img, "Gray scaled image")

    # GET IMAGE DATA
    image_data = pytesseract.image_to_data(img, output_type=pytesseract.Output.DICT, lang="kaz")

    # WORD BOXES
    k = 0
    result = []
    for i, word in enumerate(image_data['text']):
        if word != "":

            k = k + 1

            # GET WORD BORDERS
            (left, top, right, bottom) = image_data['left'][i], image_data['top'][i], image_data['left'][i] + image_data['width'][i], image_data['top'][i] + image_data['height'][i]

            # CROP WORDS
            img_croped_word = img.crop((left - word_padding, top - word_padding, right + word_padding, bottom + word_padding))

            # SAVE IMAGE WORD
            result.append(img_croped_word)
            try:
                res = split_word_to_letter("temp.jpg", letter_padding)
                plot_images(res)
            except:
                continue
    return result

In [7]:
from tensorflow.keras import load_model
import cv2
import json

model2.load_weights('weights_Kazakh_letters_VGG16_CapsNets.h5')
labels = json.load('labels.json')

def detect_text(images):
    image_array = cv2.imread(os.listdir(images))
    out_value = model2.predict(image_array)
    text_output = labels[out_value]
    text = [labels[out_value] for out_value in text_output]
    return text

In [8]:
bot = telebot.TeleBot(BOT_TOKEN)

In [9]:
def processPhotoMessage(message):
    fileID = message.photo[-1].file_id
    file = bot.get_file(fileID)
    print(file)
    image_url = 'https://api.telegram.org/file/bot{}/{}'.format(BOT_TOKEN, file.file_path)
    download_image(image_url, file.file_path)
    text = detect_text(file.file_path)
    preprocessed_images = preprocess_photo(file.file_path)
    text = detect_text(preprocessed_images)
    print (text)
    return text

def download_image(image_url, file_path):
    datatowrite = urllib2.urlopen(image_url).read()
    Path(os.path.dirname(file_path)).mkdir(parents=True, exist_ok=True)
    with open(file_path, 'wb') as f:
        f.write(datatowrite)

In [10]:
@bot.message_handler(content_types=['photo'])
def repeat_all_messages(message): # Название функции не играет никакой роли, в принципе
    text = processPhotoMessage(message)
    bot.send_message(message.chat.id, text)

@bot.message_handler(content_types=['text'])
def send_message(message):
    bot.send_message(message.chat.id, start_message_text)

@bot.message_handler(commands=['start', 'help'])
def send_message(message):
	bot.send_message(message.chat.id, start_message_text)

In [ ]:
bot.polling(none_stop=True)

{'file_id': 'AgACAgIAAxkBAANfYKi9VcMjjgfFu1Kw1cQDJA2nQAADSrYxGzvBQUmCqqLPhp0nG3RxF5suAAMBAAMCAAN4AAMWJAYAAR8E', 'file_unique_id': 'AQADdHEXmy4AAxYkBgAB', 'file_size': 21344, 'file_path': 'photos/file_6.jpg'}
Texts:

"1. Бірінші Сөз
Бұл жасқа келгенше жаксы өткіздік не, жаман өткілік не, әйтеуір бірталай
омірімізді өткізік: алыстык, жұлыстык, айтыстык, тартыстық әурешілікті,
коре-коре келдік. Енді жер ортасы жасқа келдік: калжыдык, жалыктык кылып
жүрген ісіміздің бәрінін баянқызын, багілиусыгын көрдік, бәрі қоршылық
екенін білдік. Ал, енді қалған өмірімізді қайтіп, не кылып откіземіз? Соны таба
алмаt еім де кайранмын.
Ел бағу? Жок, елге бағым жок. Баг усыз дертке ұшырайын деген кісі
бактаса, не абарткан, косілі басылмаған жастар бағамын демесе, білі Құлаii
сақтасын!
Мал балу? Жок, бага алмаймын. Балалар еклеріне керегінне веруі балар,
Енді картаа анда кылагын озілі түгел қоре алмайтұғын, ұры, залам,
тілімсектердің азыгын багын беремін деп, калган аз ғана емiрiмдi кор кылар
жайым жос.
Гы